In [4]:
import numpy as np
import scipy.signal as signal
from scipy.stats import skew as skewness
from scipy.stats import kurtosis as kurtosis_value
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import librosa 
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.layers import LSTM, Dense, Dropout
from keras.models import Sequential
from keras.regularizers import l2


dataset_audio = pd.read_csv("dataset heartbeat.csv") 
audio_paths = []
labels = []

for i in range(len(dataset_audio)):
    file_name = "dataset/training-a/wav/" + dataset_audio['Data'][i] + ".wav"
    audio_paths.append(file_name)

    print(file_name)

    # labels.append(dataset_audio['Column2'][i])
    labels.append(dataset_audio['Label'][i])



# Pra-pemrosesan audio dan pemotongan segmen
def preprocess_audio(audio_path, segment_length, overlap):
    audio, sr = librosa.load(audio_path, sr=None)
    audio_normalized = audio / np.max(np.abs(audio))  # Normalisasi amplitudo audio

    segments = []
    step = int(segment_length * (1 - overlap))
    for i in range(0, len(audio_normalized) - segment_length, step):
        segment = audio_normalized[i:i + segment_length]
        segments.append(segment)
    return segments


# Fungsi untuk menghitung fitur-fitur Hjorth
def hjorth_features(signal):
    diff1 = np.diff(signal)
    diff2 = np.diff(signal, 2)
    var_signal = np.var(signal)
    var_diff1 = np.var(diff1)
    var_diff2 = np.var(diff2)
    mobility = np.sqrt(var_diff1 / var_signal)
    complexity = np.sqrt(var_diff2 / var_diff1) / mobility if var_diff1 != 0 else 0
    return np.mean(signal), np.std(signal), mobility, complexity, np.mean(diff1), np.std(diff1), np.mean(diff2), np.std(diff2)


# Pra-pemrosesan audio dan ekstraksi fitur Hjorth
segment_length = 1000  # Panjang segmen suara (misalnya 1 detik)
overlap = 0.5  # Jumlah tumpang tindih antara segmen-suara
features = []


for audio_path in audio_paths:
    audio, sr = librosa.load(audio_path, sr=None)
    audio_normalized = audio / np.max(np.abs(audio))  # Normalisasi amplitudo audio
    mean, std, mobility, complexity, mean_diff1, std_diff1, mean_diff2, std_diff2 = hjorth_features(audio_normalized)
    features.append([mean, std, mobility, complexity, mean_diff1, std_diff1, mean_diff2, std_diff2])



features = np.array(features)
labels = np.array(labels)

# Melakukan label encoding pada label kelas
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)



# # Split dataset menjadi data latih dan data uji
# X_train, X_test, y_train, y_test = train_test_split(features, labels_encoded, test_size=0.2, random_state=42)

# # Mengubah bentuk data menjadi 3 dimensi untuk RNN-LSTM (jumlah sampel, panjang setiap sampel, jumlah fitur)
# X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
# X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

x_train, x_test, y_train, y_test = train_test_split(features, labels_encoded, test_size=0.2, random_state=42)


# x_train = np.concatenate((normal_sounds, murmur_sounds))
# y_train = np.concatenate((normal_labels, murmur_labels))
x_train[5].shape

# Mengubah bentuk data menjadi 3 dimensi untuk RNN-LSTM (jumlah sampel, panjang setiap sampel, jumlah fitur)
x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))

x_train[5].shape
print('Build LSTM RNN model ...')
model = Sequential()
model.add(LSTM(units=64, dropout=0.05, recurrent_dropout=0.35, return_sequences=True,input_shape = (1, 8)))
model.add(LSTM(units=32, dropout=0.05, recurrent_dropout=0.35, return_sequences=False))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=32, epochs=100)

model.summary()

# Melakukan prediksi pada data uji
predictions = model.predict(x_test)
y_pred = np.argmax(predictions, axis=1)

# Mengembalikan label kelas dari label encoding
y_test = label_encoder.inverse_transform(y_test)
y_pred = label_encoder.inverse_transform(y_pred)

# Menghitung akurasi
accuracy = accuracy_score(y_test, y_pred)
print("Akurasi:", accuracy)

FileNotFoundError: [Errno 2] No such file or directory: 'dataset heartbeat.csv'